<a href="https://colab.research.google.com/github/missaixxka/HomeworkPython/blob/main/HWTest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Разработка программы тестирования на Python
Цель:

Разработать консольную программу для проведения тестирования, которая позволяет загружать вопросы из файла, перемешивать их и варианты ответов, проводить тестирование пользователя, сохранять результаты в файл и предоставлять статистику по выполнению теста. Программа должна быть удобной для пользователя, с четкими инструкциями и немедленной обратной связью после каждого ответа.

In [ ]:
import datetime
import random

class Question:
    def __init__(self, text, options, correct_answer_n):
        self.text = text # Текст вопроса
        self.options = options # Массив вариантов ответа
        self.correct_answer_n = correct_answer_n  # Номер правильного ответа

    def shuffle_options(self): # Перемешивает варианты ответов
        correct_answer_text = self.options[self.correct_answer_n - 1]  # Получаем текст правильного ответа(отнимаем от номера 1, так как индексы в массиве начинаются с 0)
        random.shuffle(self.options)
        # Обновляем номер правильного ответа после перемешивания
        self.correct_answer_n = self.options.index(correct_answer_text) + 1

    def check_answer(self, user_answer): # Проверяет, правильный ли ответ дал пользователь(он должен отвечать номерами ответов!)
        return user_answer == self.correct_answer_n

def load_questions(filename): # Загружает вопросы c ответами из текстового файла
    questions = []
    try:
        with open(filename, 'r', encoding='utf-8') as f:
            for line in f:
                parts = line.strip().split('|') # .strip() - это метод, который удаляет пробельные символы в начале и в конце строки
                if len(parts) == 7: # Частей должно получится ровно  7
                    text = parts[0] # Текст вопроса
                    options = parts[1:6] # Массив вариантов ответа
                    correct_answer_text = parts[6] # Текст правильного ответа

                    # Находим номер правильного ответа в списке вариантов
                    try:
                        correct_answer_num = options.index(correct_answer_text) + 1  # Номера начинаются с 1
                    except ValueError:
                        print(f"Ошибка: Правильный ответ '{correct_answer_text}' не найден в вариантах для вопроса: {text}")
                        continue # Пропускаем этот вопрос

                    questions.append(Question(text, options, correct_answer_num))
                else:
                    print(f"Предупреждение: Неверный формат строки в файле вопросов: {line.strip()}")

    except FileNotFoundError:
        print(f"Ошибка: Файл '{filename}' не найден.")
        return None
    except Exception as e:
        print(f"Ошибка при чтении файла: {e}")
        return None

    return questions # Возвращаем массив вопросов с ответами(объекты класса Question)

def testing(questions): # Проводит тестирование пользователя.
    start_time = datetime.datetime.now() # Записываем время начала тестирования.
    correct_answers = 0 # Счетчик правильных ответов.
    total_questions = len(questions)

    random.shuffle(questions)  # Перемешиваем вопросы c ответами
    '''
    enumerate() - это встроенная функция в Python.
    Она принимает последовательность (например, список, кортеж или строку)
    в качестве аргумента и возвращает объект-итератор,
    который генерирует последовательность кортежей вида (index, element).
    '''
    for i, question in enumerate(questions): # Выводит вопросы с ответами на консоль
        question.shuffle_options()  # Перемешиваем варианты ответов
        print(f"Вопрос {i + 1}/{total_questions}: {question.text}")
        for j, option in enumerate(question.options):
            print(f"{j + 1}. {option}")

        while True: # Бесконечный цикл, который будет продолжаться до тех пор, пока пользователь не введет корректный ответ (это должен быть именно номер ответа(1-5))
            try:
                user_answer = int(input("Ваш ответ (1-5): "))
                if 1 <= user_answer <= 5:
                    break
                else:
                    print("Пожалуйста, введите число от 1 до 5.")
            except ValueError:
                print("Пожалуйста, введите число.")

        if question.check_answer(user_answer):
            print("Правильно!")
            correct_answers += 1
        else:
            print("Неправильно.")

        if i != (total_questions-1):
            print("Переходим к следующему вопросу...")
        print() # Выводим пустую строку для визуального разделения вопросов

    end_time = datetime.datetime.now() # Записываем время конца тестирования
    return start_time, end_time, correct_answers, total_questions

def save_results_in_file(filename, start_time, end_time, correct_answers, total_questions): # Сохраняет результаты тестирования в файл
    try:
        with open(filename, 'w', encoding='utf-8') as f:
            f.write(f"Время начала теста: {start_time}\n")
            f.write(f"Время окончания теста: {end_time}\n")
            f.write(f"Общее количество вопросов: {total_questions}\n")
            f.write(f"Количество правильных ответов: {correct_answers}\n")
            if total_questions > 0:
                percentage = (correct_answers / total_questions) * 100
            else:
                percentage = 0
            f.write(f"Процент правильных ответов: {percentage:.2f}%\n")
        print(f"Результаты сохранены в файл: {filename}")
    except Exception as e:
        print(f"Ошибка при сохранении результатов в файл: {e}")

def main(): # Основная функция программы
    questionss = load_questions("questions.txt")
    if questionss is None:
        return  # Завершаем программу, если не удалось загрузить вопросы

    if not questionss:
        print("В файле нет вопросов или они имеют неверный формат. Проверьте файл questions.txt")
        return

    print("Добро пожаловать в программу тестирования!")

    start_time, end_time, correct_answers, total_questions = testing(questionss)

    print("Тестирование завершено!")
    print(f"Общее количество вопросов: {total_questions}")
    print(f"Количество правильных ответов: {correct_answers}")
    if total_questions > 0:
        percentage = (correct_answers / total_questions) * 100
    else:
        percentage = 0
    print(f"Процент правильных ответов: {percentage:.2f}%")

    save_results_in_file("results.txt", start_time, end_time, correct_answers, total_questions)

main()
print("Программа завершена!")


# questions.txt
Какой язык программирования чаще всего используется для машинного обучения?|Python|Java|C++|JavaScript|R|Python
Какой год считается годом основания Москвы?|1147|1054|1237|1704|988|1147
Кто написал пьесу 'Гамлет'?|Уильям Шекспир|Лев Толстой|Федор Достоевский|Антон Чехов|Джейн Остин|Уильям Шекспир
Какая планета в нашей Солнечной системе самая большая?|Сатурн|Юпитер|Земля|Марс|Венера|Юпитер
Что такое SQL?|Язык запросов к базам данных|Операционная система|Вид видеоигры|Тип процессора|Метод шифрования|Язык запросов к базам данных
Какой химический символ у золота?|Ag|Ca|Au|K|O|Au
Кто является автором теории относительности?|Исаак Ньютон|Галилео Галилей|Нильс Бор|Альберт Эйнштейн|Стивен Хокинг|Альберт Эйнштейн
Какая столица Австралии?|Канберра|Сидней|Мельбурн|Брисбен|Перт|Канберра
Какой газ составляет большую часть атмосферы Земли?|Азот|Углекислый газ|Кислород|Аргон|Водород|Азот
Кто был первым человеком, ступившим на Луну?|Юрий Гагарин|Эдвин Олдрин|Нил Армстронг|Валентина Терешкова|Алексей Леонов|Нил Армстронг